<center>
<h1>LSTM básico: Haciendo N-gramas</h1>


<p> Julio Waissman Vilanova </p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/pln/blob/main/labs/RNN/deep-ngram.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

Tomado parcialmente y adaptado de [el repositorio de github](https://github.com/shaundsouza/lstm-textual-ngrams) del trabajo [*LSTM Neural Network for Textual Ngrams* (D'Souza, 2018)](https://www.preprints.org/manuscript/201811.0579/v1)



</center>


In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

## Importando datos de alguna obra 

Vamos a descargar un *Corpus*, y pues vamos a usar algo muy famoso, como  puede ser la obra de Shakespiare en inglés, o *El Quijote* en español. Empecemos por *El Quijote*

In [2]:
!curl -o quijote.txt https://www.gutenberg.org/cache/epub/2000/pg2000.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
 30 2173k   30  661k    0     0   279k      0  0:00:07  0:00:02  0:00:05  280k
100 2173k  100 2173k    0     0   800k      0  0:00:02  0:00:02 --:--:--  801k


y ahora vamos a descargar el corpus por lineas de texto, eliminando cabecera y final de Gutemberg, quitando espacios, moviendo todas las letras a minúsculas.

In [3]:
archivo = "quijote.txt"
corpus = []
with open(archivo, 'r', encoding='utf8') as fp:
    guarda = False
    for line in fp.readlines():
      if "*** END OF THE PROJECT GUTENBERG" in line:
        break
      if guarda and len(line) > 2:
        corpus.append(line.strip())
      if "*** START OF THE PROJECT GUTENBERG" in line:
        guarda = True

Veamos como de ve el corpus:

In [4]:
print(f"Inicio del texto: \n{corpus[:10]}")

Inicio del texto: 
['El ingenioso hidalgo don Quijote de la Mancha', 'por Miguel de Cervantes Saavedra', 'El ingenioso hidalgo don Quijote de la Mancha', '', 'Tasa', '', 'Testimonio de las erratas', '', 'El Rey', '']


In [5]:
print(f"Fin del texto: \n{corpus[-10:]}")

Fin del texto: 
['para hacer burla de tantas como hicieron tantos andantes caballeros, bastan', 'las dos que él hizo, tan a gusto y beneplácito de las gentes a cuya noticia', "llegaron, así en éstos como en los estraños reinos''. Y con esto cumplirás", 'con tu cristiana profesión, aconsejando bien a quien mal te quiere, y yo', 'quedaré satisfecho y ufano de haber sido el primero que gozó el fruto de', 'sus escritos enteramente, como deseaba, pues no ha sido otro mi deseo que', 'poner en aborrecimiento de los hombres las fingidas y disparatadas', 'historias de los libros de caballerías, que, por las de mi verdadero don', 'Quijote, van ya tropezando, y han de caer del todo, sin duda alguna. Vale.', 'Fin']


In [6]:
print(type(corpus))
print(len(corpus))

<class 'list'>
31941


## Preprocesando la información 

Para el preprocesamiento, vamos a convertir a tokens cada linea de texto y vamos a agregar pads al mas largo de los textos que se encuentran en el documento (antes de cada salto de linea). Vamos a utilizar la capa de `TextVectorization` que ofrece `Keras`:

In [7]:
max_tokens = 10_000

indizador = layers.TextVectorization(max_tokens=max_tokens)
indizador.adapt(corpus)

Por lo que tenemos un nuevo vocabulario que podemos ver en orden:

In [8]:
vocab = indizador.get_vocabulary()
print(f"Tenemos un vocabulario de {len(vocab)} tokens")
print(f"Y aquí están las primeras 20 palabras:\n{vocab[:20]}")

Tenemos un vocabulario de 10000 tokens
Y aquí están las primeras 20 palabras:
['', '[UNK]', 'que', 'de', 'y', 'la', 'a', 'el', 'en', 'no', 'los', 'se', 'con', 'por', 'lo', 'las', 'le', 'su', '—', 'don']


Y ahora vamos a convertir el *corpus* en tensores de tokens simplemente como:

In [9]:
entradas = indizador(corpus)

print(f"Las entradas en un tensor con un shape de:\n{entradas.shape}")

Las entradas en un tensor con un shape de:
(31941, 20)


Como podemos ver tenemos una serie de tokens por linea (o muestra) que se generó del *corpus*. Ahora vamos poniendo la secuencia de salida. 

Como lo que queremos es estimar la próxima palabra, pues no queda mas que usar los mismos tokens de entrada, pero con un corrimiento hacia la izquierda:

In [10]:
salidas = tf.roll(entradas, shift=-1, axis=1)

print(f"Entrada 1,000: \n{entradas[1_000]}")
print(f"Salida 1,000: \n{salidas[1_000]}")

Entrada 1,000: 
[ 574  901 2131    3    1    8 6373    1    8 5825    4    0    0    0
    0    0    0    0    0    0]
Salida 1,000: 
[ 901 2131    3    1    8 6373    1    8 5825    4    0    0    0    0
    0    0    0    0    0  574]


Como podemos ver, las salidas son iguales a las entradas pero con un adelanto, así ya podemos especificar nuestro modelo de N-gramas.

# Modelo y entrenamiento

El modelo es muy sencillo y podemos discutirlo mucho y modificarlo para probar nuevas cosas.